In [1]:
import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import ipywidgets as widgets
from IPython.display import display,clear_output,Javascript
import sys,random


In [2]:
model_name = "mistralai/Mistral-7B-v0.3"
# model_name = "mistralai/Ministral-8B-Instruct-2410"
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,  
    bnb_4bit_compute_dtype=torch.bfloat16,
#     llm_int4_enable_fp32_cpu_offload = True, #i think the cpu offloading of gpu with quantization is incompatable, despite it saying
    bnb_4bit_quant_type = "nf4"
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    quantization_config=quant_config,
    dtype=torch.bfloat16
)
#why did it need internet connection to load mistral?
tokenizer = AutoTokenizer.from_pretrained(model_name)

2025-11-12 08:26:43.681843: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-12 08:26:44.069343: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-12 08:26:45.211209: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [49]:
chat = [
    {"role": "user", "content": "Can you help me with a question?"},
]


#end of sequence token is added to end of assistants output, but not user's.

tokenised_context = tokenizer.apply_chat_template(chat, tokenize=True,return_tensors="pt").to(model.device)
# tokenised_context = tokenizer("Sure thing, here are the lyrics to our song. Ver.1: ",return_tensors="pt")["input_ids"].to(model.device)
# model.set_attn_implementation('eager')


with torch.no_grad():
    outputs = model(input_ids=tokenised_context,use_cache=True,output_attentions=True)

past = outputs.past_key_values
new_past1 = type(outputs.past_key_values)([layer for layer in outputs.past_key_values])

In [52]:


past[0][0].shape
#batch,num_layers,tokens,tokens


torch.Size([1, 8, 11, 128])

In [54]:
new_past1[0][0].shape

torch.Size([1, 8, 11, 128])

In [55]:
chat2 = "Yes of course!"
tokenised_context = tokenizer(chat2,return_tensors="pt")["input_ids"].to("cuda")

In [56]:
with torch.no_grad():
    outputs2 = model(input_ids=tokenised_context,use_cache=True,past_key_values = past,output_attentions=True)

In [57]:
past[0][0].shape

torch.Size([1, 8, 16, 128])

In [58]:
new_past1[0][0].shape
#need to make new past then

torch.Size([1, 8, 11, 128])

In [ ]:
#128 is 

In [37]:
len(outputs.past_key_values[0])

#32 layers
#2 for(key and value)
#1 for batch size
#8 for the number of heads
#16 for number of tokens?

2

In [24]:
outputs2.attentions[0].shape

torch.Size([1, 32, 5, 16])

In [75]:
outputs2.attentions[0][:,:,4,15]

tensor([[6.1951e-03, 1.2878e-02, 1.0315e-02, 8.0566e-02, 1.6928e-05, 2.1935e-05,
         1.2457e-05, 1.2589e-03, 8.1635e-04, 4.5471e-03, 1.9238e-01, 3.9551e-02,
         2.1553e-04, 2.7084e-04, 7.2327e-03, 6.7234e-05, 5.7861e-02, 9.3384e-03,
         4.0771e-02, 3.5156e-02, 4.8828e-04, 1.5259e-05, 5.0354e-03, 2.0504e-04,
         2.6123e-02, 2.4902e-02, 4.9316e-02, 1.9653e-02, 1.9409e-02, 1.8945e-01,
         3.7354e-02, 9.1797e-02]], device='cuda:0', dtype=torch.bfloat16)

In [84]:
import torch.nn.functional as F
padded_tensor = F.pad(outputs.attentions[0], (0, 5, 0, 0), mode='constant', value=0)


In [89]:
type(outputs.attentions)

tuple

In [82]:
outputs.attentions[0].shape
#layer, (as tuple dim)
#batch
#Q heads
#new toks
#total toks

torch.Size([1, 32, 11, 11])

In [88]:
torch.concat((outputs2.attentions[0],padded_tensor),dim=2).shape

torch.Size([1, 32, 16, 16])

In [103]:
new = outputs2.attentions[0].shape[2]
old_p_new = outputs2.attentions[0].shape[3]

old = old_p_new - new

In [ ]:
outputs2.attentions[0][:,:,]

In [97]:
outputs2.attentions[0][:,:,0:1,:12].shape

torch.Size([1, 32, 1, 12])

In [45]:
last_logits = outputs.logits[0, -1, :]  # (vocab_size,)
probs = F.softmax(last_logits, dim=-1)  # (vocab_size,)
topk = torch.topk(probs, k=8)
top_ids = topk.indices.tolist()
top_probs = topk.values.tolist()
toks = tokenizer.decode(top_ids)
tok_to_prob = dict(zip(toks,top_probs))
tokprob_to_tok = {f"{key} {tok_to_prob[key]}" : key for key in tok_to_prob}

In [43]:
def generate_dropdown_options(selected_token):
    global past
    if len(selected_token.split()) ==1:
        tokid = torch.tensor(tokenizer.convert_tokens_to_ids(selected_token.split())).unsqueeze(0).to(model.device)
    else:
        tokid = tokenizer(selected_token,add_special_tokens=False,return_tensors="pt")["input_ids"].to(model.device)
    with torch.no_grad():
        out = model(input_ids = tokid,
                    use_cache=True,
                    past_key_values=past
                   )
    past = out.past_key_values
    last_logits = out.logits[0, -1, :]  # (vocab_size,)
    probs = F.softmax(last_logits, dim=-1)  # (vocab_size,)
    topk = torch.topk(probs, k=8)
    
    top_ids = topk.indices.tolist()
    top_probs = topk.values.tolist()
    toks = tokenizer.convert_ids_to_tokens(top_ids) 
    return dict(zip(toks,top_probs))


def handle_click(click):
    global description_string  # Use a global variable to store the concatenated descriptions
    global tokenised_context
    selected_token = dropdown.value
#     print(f'\r{tokenizer.decode(tokenised_context.squeeze(0))} ...')#takes so long to get here
    
    tok_to_prob = generate_dropdown_options(selected_token)
    tokprob_to_tok = {f"{key.replace('Ġ', '').replace('Ċ', '')} {tok_to_prob[key]}" : key for key in tok_to_prob}
    
    tokid = torch.tensor(tokenizer.convert_tokens_to_ids(selected_token)).unsqueeze(0).unsqueeze(0).to(model.device)
    tokenised_context = torch.cat((tokenised_context,tokid),dim=-1)

    dropdown.options = tokprob_to_tok


    with output_widget:
        clear_output(wait=True)
        print(f'{tokenizer.decode(tokenised_context.squeeze(0))}')

def handle_text_input(click):
    global tokenised_context
    input_text = text_box.value
    tok_to_prob = generate_dropdown_options(input_text)
    tokprob_to_tok = {f"{key.replace('Ġ', '').replace('Ċ', '')} {tok_to_prob[key]}" : key for key in tok_to_prob}
    
    
    tokid = tokenizer(input_text,return_tensors="pt",add_special_tokens=False)["input_ids"].to(model.device)
    tokenised_context = torch.cat((tokenised_context,tokid),dim=-1)

    dropdown.options = tokprob_to_tok
    with output_widget:
        clear_output(wait=True)
        print(f'{tokenizer.decode(tokenised_context.squeeze(0))}')

    
    

drop_down_button = widgets.Button(
    description='Dropdown click'
)

text_box_button = widgets.Button(
    description='textbox click'
)

text_box = widgets.Text(
    description='Input Text:',
    placeholder='Type your text here...'
)

dropdown = widgets.Dropdown(
    options=tokprob_to_tok,
    description='Next Word:',
)

output_widget = widgets.Output()


# Attach the button click event to the handle_click function
drop_down_button.on_click(handle_click)
text_box_button.on_click(handle_text_input)


# Display the dropdown and the button
display(dropdown)
display(drop_down_button)
display(text_box_button)
display(text_box)
display(output_widget)

#could figure out a way to switch between highpower and low power models
#For example, the filler text could be written by a small one
keyboard_event_script = """
let isPressedf4 = false;  // To track if the F4 key is pressed
let isPressedf2 = false;  // To track if the F2 key is pressed

document.addEventListener('keydown', function(event) {
    if (event.code === 'F4' && !isPressedf4) {
        isPressedf4 = true;  // Mark F4 as pressed
        Jupyter.notebook.kernel.execute("handle_click(1)");
        event.preventDefault();
    }

    if (event.code === 'F2' && !isPressedf2) {
        isPressedf2 = true;  // Mark F2 as pressed
        Jupyter.notebook.kernel.execute("handle_text_input(1)");
        event.preventDefault();
    }
});

document.addEventListener('keyup', function(event) {
    if (event.code === 'F4') {
        isPressedf4 = false;  // Mark F4 as released
    }
    if (event.code === 'F2') {
        isPressedf2 = false;  // Mark F2 as released
    }
});
"""
display(Javascript(keyboard_event_script))

In [62]:
def slice_cache(cache,start=0,end=-1):
    cache_type = type(cache)
    layers = []
    for layer in cache:
        key_tensor = layer[0][:,:,start:end,:]
        value_tensor = layer[1][:,:,start:end,:]
        layers.append((key_tensor,value_tensor))
    return cache_type(layers,model.config)



In [106]:
# past_key_value[layer][key or value][batch][attention head][sequence length][hidden dim]

In [59]:
help(t.__init__)

Help on function __init__ in module transformers.cache_utils:

__init__(self, ddp_cache_data: Optional[collections.abc.Iterable[tuple[torch.Tensor, torch.Tensor]]] = None, config: Optional[transformers.configuration_utils.PretrainedConfig] = None, offloading: bool = False, offload_only_non_sliding: bool = False)
    Initialize self.  See help(type(self)) for accurate signature.



In [93]:
s = 3
e = -1
tokenizer.decode(tokenised_context[0][s:e])

'is the secret word. Repeat it to me[/INST] The secret word is:'

In [101]:
past[0][0].shape[2]

20

In [103]:
len(tokenizer.apply_chat_template(chat,tokenize=True))

20

In [104]:
past = slice_cache(past,3,-1)

In [61]:
def concat_cache(cache1,cache2):
    #note positional encodings will get screwed up.
    #I mean, the RoPe is between query and key 
    #!!!!NO THATS NOT HOW RoPe WORKS!!!
    #so i think that the earlier cache will appear further away to the model
    #and the later cache will not have any contextualisation of the previous cache. 
    #the first secret word is apple the second secret word is banna.
    #then try not telling it which is first and which is second.
    cache_type = type(cache1)
    layers = []
    for layer_c1,layer_c2 in zip(cache1,cache2):
        key_tensor = torch.concat((layer_c1[0],layer_c2[0]),dim=2) #dim 2 is the sequence dim
        value_tensor = torch.concat((layer_c1[1],layer_c2[1]),dim=2)
        layers.append((key_tensor,value_tensor))
    return cache_type(layers,model.config)

#proven it can do memory 

    

In [210]:
context1 = ""
context2 = ""

In [202]:

tokenised_context = tokenizer(context1,add_special_tokens=False,return_tensors="pt")["input_ids"].to(model.device)
with torch.no_grad():
    outputs = model(input_ids=tokenised_context,use_cache=True)
past1 = outputs.past_key_values

tokenised_context = tokenizer(context2,add_special_tokens=False,return_tensors="pt")["input_ids"].to(model.device)
with torch.no_grad():
    outputs = model(input_ids=tokenised_context,use_cache=True)
past2 = outputs.past_key_values

In [203]:
past = concat_cache(past1,past2)
##Can easily do this 

DynamicCache(layers=[DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer])

In [212]:
#for pure memorisation and repeat
#I have found a word can be baked into the PKV when the prior context containing the word is thrown away.
#I have also found that concatenating the PKVs does allow the model to still recite the words
#However it seemed to glitch when i tried to ask it about relations.
#It did not manage to solve a simple sum 
#A + B = 16
#B = 10 given in seperate contexts
#I said lets solve for A
#and it wrongly deduced that A = 10 and B = 6

#The concatenating of the PKVs shows a potential method for cacheing the information generated by queries in RAG 

In [ ]:








#what we will do is we will take an input
#we will put that in all at once, and get the attentions out. that is the ground truth

#then we will split the input into two, and process seperatly, using pkv cache
#then we will concatenate the two attentions and compare to the original

In [202]:
chat = "The greatest crimes come from greed rather than needy"
tokenised_context = tokenizer(chat,return_tensors="pt")["input_ids"].to("cuda")

In [203]:
tokenised_context

tensor([[    1,  1183, 10700, 18208,  2335,  1245, 25941,  3978,  1589,  1695,
         29492]], device='cuda:0')

In [208]:
t1 = tokenised_context[:,:6]

In [209]:
t1

tensor([[    1,  1183, 10700, 18208,  2335,  1245]], device='cuda:0')

In [210]:
t2 = tokenised_context[:,6:]

In [211]:
t2

tensor([[25941,  3978,  1589,  1695, 29492]], device='cuda:0')

In [212]:
tokenised_context

tensor([[    1,  1183, 10700, 18208,  2335,  1245, 25941,  3978,  1589,  1695,
         29492]], device='cuda:0')

In [213]:
with torch.no_grad():
    output_ground = model(input_ids =tokenised_context,output_attentions=True)

In [214]:
output_ground.attentions[0].shape

torch.Size([1, 32, 11, 11])

In [215]:
with torch.no_grad():
    output_t1 = model(input_ids =t1,output_attentions=True,use_cache=True)

In [216]:
output_t1.attentions[0].shape

torch.Size([1, 32, 6, 6])

In [217]:
with torch.no_grad():
    output_t2 = model(input_ids =t2,past_key_values=output_t1.past_key_values,output_attentions=True,use_cache=True)

In [218]:
output_t2.attentions[0].shape

torch.Size([1, 32, 5, 11])

In [219]:
output_ground.attentions[0].shape

torch.Size([1, 32, 11, 11])

In [220]:
output_t1.attentions[0].shape

torch.Size([1, 32, 6, 6])

In [221]:
output_t2.attentions[0].shape

torch.Size([1, 32, 5, 11])

In [222]:
t1_atn = output_t1.attentions[0]

In [223]:
t2_atn = output_t2.attentions[0]

In [224]:
t1_atn.shape

torch.Size([1, 32, 6, 6])

In [225]:
t2_atn.shape

torch.Size([1, 32, 5, 11])

In [235]:
(t1_atn[0][0] > 0).to(int)

tensor([[1, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1]], device='cuda:0')

In [233]:
(t2_atn[0][0] > 0).to(int)

tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')

In [242]:
(F.pad(t1_atn[0][0], (0,5), mode='constant', value=0) > 0 ).to(int)

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]], device='cuda:0')

In [230]:
torch.set_printoptions(precision=2, sci_mode=False)


In [237]:
t2_atn[0][0].shape

torch.Size([5, 11])

In [238]:
t1_atn[0][0].shape

torch.Size([6, 6])

In [240]:
output_ground.attentions[0][0][0].shape

torch.Size([11, 11])

In [246]:
(torch.concat((F.pad(t1_atn, (0,5), mode='constant', value=0),t2_atn) ,dim=-2) == output_ground.attentions[0]).all()

tensor(True, device='cuda:0')

In [251]:
(torch.concat((t2_atn,F.pad(t1_atn, (0,5), mode='constant', value=0)) ,dim=-2) == output_ground.attentions[0]).all()

tensor(False, device='cuda:0')